<a href="https://colab.research.google.com/github/doris-asongafac/Support-Vector-Machines/blob/main/emotion_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from transformers import pipeline
from sklearn.metrics import accuracy_score, classification_report
import torch

#Check for CUDA
device = 0 if torch.cuda.is_available() else -1
device_name = torch.cuda.get_device_name(0) if device == 0 else "CPU"
print(f"Using device: {device_name}(device={device})")
from google.colab import files
uploaded = files.upload()


Using device: CPU(device=-1)


Saving emotion_analysis_results.csv to emotion_analysis_results (1).csv


In [ ]:
import pandas as pd
data = pd.read_csv('emotion_analysis_results.csv')

#Initialize the emotion classification pipeline with specified device
try:
    emotion_analyzer = pipeline(
        "text-classification",
        model="j-hartmann/emotion-english-distilroberta-base",
        device=0
    )
except Exception as e:
    print(f"Error initializing the pipeline: {e}")
    exit(1)

# Function to predict emotion and map to Final Emotion labels
def predict_emotion(review):
    try:
        # Truncate review to max length (DistilBERT handles ~512 tokens)
        max_length = 512
        review = review[:max_length] if len(review) > max_length else review
        result = emotion_analyzer(review)[0]
        predicted_label = result['label'].lower()

        # Map model labels to Final Emotion labels
        label_mapping = {
            'joy': 'happiness',
            'sadness': 'sadness',
            'anger': 'anger',
            'disgust': 'disgust',
            'surprise': 'surprise',
            'fear': 'unknown'  # Handle fear as unknown since it's not in the dataset
        }

        return label_mapping.get(predicted_label, 'unknown')

    except Exception as e:
        print(f"Error processing review: {e}")
        return 'unknown'

#Apply emotion prediction to each review
data['predicted_emotion'] = data['review'].apply(predict_emotion)

#Standardize Final Emotion column for consistency
data['Final Emotion'] = data['Final Emotion'].str.lower()

#Compare manual and predicted emotions
data['match'] = data['Final Emotion'] == data['predicted_emotion']

#Calculate accuracy
valid_predictions = data[data['predicted_emotion'] != 'unknown']
accuracy = accuracy_score(valid_predictions['Final Emotion'], valid_predictions['predicted_emotion'])
print(f"Accuracy: {accuracy:.2f}%")

# Generate classification report
print("\nClassification Report:")
labels = ['happiness', 'sadness', 'anger', 'disgust', 'surprise']
print(classification_report(valid_predictions['Final Emotion'], valid_predictions['predicted_emotion'], labels=labels))

# Save results to a new CSV
output_file = 'emotion_predictions.csv'
data[['review', 'Final Emotion', 'predicted_emotion', 'match']].to_csv(output_file, index=False)
print(f"\nResults saved to {output_file}")

# Display a sample of the results
print("\nSample of Results:")
print(data[['review', 'Final Emotion', 'predicted_emotion', 'match']].head(10))

# Save a summary report
summary_report = f""" Emotion Analysis Summary
====================================
Model Used: bhadesh-savani/distilbert-base-uncased-emotion
Device: {device_name} (device={device})
Total Reviews: {len(data)}
Accuracy: {accuracy:.2f}%
Output File: {output_file}

Classification Report:
{classification_report(valid_predictions['Final Emotion'], valid_predictions['predicted_emotion'], labels=labels)}
"""

with open('emotion_analysis_summary.txt', 'w') as f:
  f.write(summary_report)
print("\nSummary report saved to emotion_analysis_summary.txt")


Device set to use cpu


Accuracy: 0.43%

Classification Report:
              precision    recall  f1-score   support

   happiness       0.76      0.72      0.74        18
     sadness       0.00      0.00      0.00         0
       anger       0.00      0.00      0.00         9
     disgust       0.58      0.54      0.56        28
    surprise       0.09      1.00      0.17         1

   micro avg       0.43      0.52      0.47        56
   macro avg       0.29      0.45      0.29        56
weighted avg       0.54      0.52      0.52        56


Results saved to emotion_predictions.csv

Sample of Results:
                                              review Final Emotion  \
0  One of the other reviewers has mentioned that ...     happiness   
1  A wonderful little production. <br /><br />The...     happiness   
2  I thought this was a wonderful way to spend ti...     happiness   
3  Basically there's a family where a little boy ...       disgust   
4  Petter Mattei's "Love in the Time of Money" is...     ha

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.p